In [1]:
# Import libraries
import awswrangler as wr
import numpy as np
import pandas as pd
#%%
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np
from sklearn.metrics import mean_absolute_error
print(tf.__version__)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

2022-07-22 18:53:41.856427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-22 18:53:41.856457: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


In [2]:
# read the files 
meth_path = 's3://adisiddharth/M_batch1_batch2_select_no_ffpe_corrected.csv'
meth = wr.s3.read_csv(meth_path)

pheno_path = 's3://adisiddharth/phenotype_mappings.csv'
pheno = wr.s3.read_csv(pheno_path)

In [3]:
# Change . to _ in column headers
# Rename some columns 
# Subset the cols of interest
pheno = pheno.rename(columns={col: col.replace('.', '_') for col in pheno.columns})
pheno = pheno.rename(columns={'Unnamed: 0': 'meth_id'})
vars_of_interest = ['meth_id', 'Endometriosis__Yes_No_', 'Cycle_phase_for_Analysis', 'Endometriosis_stage_grouped__I_II____III_IV_', 'Institute_for_Analysis', 'Batch']
pheno = pheno.loc[:, vars_of_interest]
pheno = pheno.rename(columns={'Endometriosis__Yes_No_': 'Endometriosis', 'Endometriosis_stage_grouped__I_II____III_IV_': 'Stage', 'Institute_for_Analysis': 'Institute', 'Cycle_phase_for_Analysis': 'Cycle_Phase'})

In [4]:
# Transpose the meth df 
meth_t = meth.T

new_header = meth_t.iloc[0] #grab the first row for the header
meth_t = meth_t[1:] #take the data below the header row
meth_t.columns = new_header #set the header row as the df header
meth_t = meth_t.rename_axis('' ,axis = 'columns')
meth_t = meth_t.reset_index()
meth_t = meth_t.rename(columns= {'index': 'meth_id'})
meth_t.head()

df = pheno.merge(meth_t, on = 'meth_id')
df.set_index(df['meth_id'])
#df = df.drop(['meth_id'], axis = 1)

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg21870274,cg09499020,cg16535257,cg06325811,...,cg00763725,cg12403148,cg07311845,cg12115116,cg12830022,cg16667333,cg07211220,cg01566232,cg15894539,cg07551525
meth_id,,,,,,,,,,,,,,,,,,,,,
201904640089_R01C01,201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1,2.039036,-0.721799,-0.411768,-1.484742,...,0.389419,0.539956,-1.324202,0.583998,1.356319,1.159705,3.365449,-0.171601,2.604686,3.532742
201904640089_R02C01,201904640089_R02C01,No,PE,CONTROL,UCSF,1,1.960953,-0.483177,-0.821167,-1.421923,...,0.203354,0.72801,-0.953156,0.542036,0.89238,1.18257,3.92592,-0.25596,3.184186,3.365681
201904640089_R03C01,201904640089_R03C01,Yes,ESE,III-IV,UCSF,1,2.091686,-0.779545,-1.158785,-1.266587,...,-0.148426,0.378684,-1.670119,0.581792,1.251458,1.38776,4.14834,0.057897,3.17159,3.287158
201904640089_R04C01,201904640089_R04C01,Yes,MSE,I-II,"IMB, UQ",1,2.681174,-0.641505,-0.856451,-1.487919,...,0.296156,0.448996,-0.926917,0.722317,1.011427,1.122688,3.709386,-0.145001,2.930336,3.115745
201904640089_R05C01,201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1,1.765793,-1.132911,-0.136729,-1.686892,...,0.184888,0.756678,-1.051571,0.358237,1.28598,1.502455,4.124252,-0.097553,3.265681,3.293867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202184900076_R03C01,202184900076_R03C01,Yes,MSE,I-II,Oxford,1,2.137374,-0.750536,-1.227665,-1.820904,...,-0.088232,0.302075,-1.377155,0.736659,1.138024,1.329556,3.873212,0.070319,3.389347,3.392994
202184900076_R04C01,202184900076_R04C01,Yes,LSE,III-IV,Oxford,1,2.927024,-0.833248,-1.561893,-0.689921,...,-0.377822,0.294278,-0.872458,0.668582,1.780835,1.256327,4.096688,-0.33414,3.277219,3.13821
202184900076_R06C01,202184900076_R06C01,Yes,LSE,I-II,Oxford,1,2.334874,-0.675145,-0.769275,-2.226923,...,1.961999,2.0729,-1.18314,0.832224,1.772527,0.597387,3.97231,-0.074274,3.015352,3.929728


In [5]:
# Group ESE, MSE and LSE into SE 
df.loc[df['Cycle_Phase'].isin(['ESE', 'MSE', 'LSE']), 'Cycle_Phase'] = 'SE'

df = df.set_index('meth_id')

# Change Cycle Phase to binary
df['Cycle_Phase_1'] = np.where(df['Cycle_Phase'] == 'PE', 0, '')
df['Cycle_Phase_2'] = np.where(df['Cycle_Phase'] == 'SE', 1, '')
df['Cycle_Phase_3'] = np.where(df['Cycle_Phase'] == 'Menstrual', 2, '')
df['Cycle_Phase_int'] = df['Cycle_Phase_1'] + df['Cycle_Phase_2'] + df['Cycle_Phase_3'] 

# Change Endo Yes/No to binary
df['Endometriosis_int'] = np.where(df['Endometriosis'] == 'Yes', 1, df['Endometriosis'])
df['Endometriosis_int'] = np.where(df['Endometriosis_int'] == 'No', 0, df['Endometriosis_int'])

# Remove MAMC samples 
mamc_index = df.loc[df['Institute'] == 'MAMC'].index
df.drop(mamc_index, inplace=True)

In [6]:
df

,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg21870274,cg09499020,cg16535257,cg06325811,cg08258224,...,cg16667333,cg07211220,cg01566232,cg15894539,cg07551525,Cycle_Phase_1,Cycle_Phase_2,Cycle_Phase_3,Cycle_Phase_int,Endometriosis_int
meth_id,,,,,,,,,,,,,,,,,,,,,
201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1,2.039036,-0.721799,-0.411768,-1.484742,3.083376,...,1.159705,3.365449,-0.171601,2.604686,3.532742,0,,,0,1
201904640089_R02C01,No,PE,CONTROL,UCSF,1,1.960953,-0.483177,-0.821167,-1.421923,3.03574,...,1.18257,3.92592,-0.25596,3.184186,3.365681,0,,,0,0
201904640089_R03C01,Yes,SE,III-IV,UCSF,1,2.091686,-0.779545,-1.158785,-1.266587,2.730783,...,1.38776,4.14834,0.057897,3.17159,3.287158,,1,,1,1
201904640089_R04C01,Yes,SE,I-II,"IMB, UQ",1,2.681174,-0.641505,-0.856451,-1.487919,2.934617,...,1.122688,3.709386,-0.145001,2.930336,3.115745,,1,,1,1
201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1,1.765793,-1.132911,-0.136729,-1.686892,3.200332,...,1.502455,4.124252,-0.097553,3.265681,3.293867,0,,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202184900076_R03C01,Yes,SE,I-II,Oxford,1,2.137374,-0.750536,-1.227665,-1.820904,2.487833,...,1.329556,3.873212,0.070319,3.389347,3.392994,,1,,1,1
202184900076_R04C01,Yes,SE,III-IV,Oxford,1,2.927024,-0.833248,-1.561893,-0.689921,2.775226,...,1.256327,4.096688,-0.33414,3.277219,3.13821,,1,,1,1
202184900076_R06C01,Yes,SE,I-II,Oxford,1,2.334874,-0.675145,-0.769275,-2.226923,3.475557,...,0.597387,3.97231,-0.074274,3.015352,3.929728,,1,,1,1


In [7]:
# seperate features and labels
y_full = df['Cycle_Phase_int']
x_full = df.iloc[:, 5:-5]

In [8]:
x_full

,cg21870274,cg09499020,cg16535257,cg06325811,cg08258224,cg16619049,cg18147296,cg13938959,cg12445832,cg23999112,...,cg00763725,cg12403148,cg07311845,cg12115116,cg12830022,cg16667333,cg07211220,cg01566232,cg15894539,cg07551525
meth_id,,,,,,,,,,,,,,,,,,,,,
201904640089_R01C01,2.039036,-0.721799,-0.411768,-1.484742,3.083376,-3.030698,1.231441,3.185491,1.122507,3.725658,...,0.389419,0.539956,-1.324202,0.583998,1.356319,1.159705,3.365449,-0.171601,2.604686,3.532742
201904640089_R02C01,1.960953,-0.483177,-0.821167,-1.421923,3.03574,-4.106582,1.183343,3.207747,1.482713,3.74316,...,0.203354,0.72801,-0.953156,0.542036,0.89238,1.18257,3.92592,-0.25596,3.184186,3.365681
201904640089_R03C01,2.091686,-0.779545,-1.158785,-1.266587,2.730783,-4.44966,0.800348,2.904233,1.102149,3.699748,...,-0.148426,0.378684,-1.670119,0.581792,1.251458,1.38776,4.14834,0.057897,3.17159,3.287158
201904640089_R04C01,2.681174,-0.641505,-0.856451,-1.487919,2.934617,-4.052979,1.488742,2.762324,1.49816,3.729866,...,0.296156,0.448996,-0.926917,0.722317,1.011427,1.122688,3.709386,-0.145001,2.930336,3.115745
201904640089_R05C01,1.765793,-1.132911,-0.136729,-1.686892,3.200332,-3.931479,1.162638,3.199668,1.421547,3.740852,...,0.184888,0.756678,-1.051571,0.358237,1.28598,1.502455,4.124252,-0.097553,3.265681,3.293867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202184900076_R03C01,2.137374,-0.750536,-1.227665,-1.820904,2.487833,-3.94961,0.917199,3.480793,2.041851,3.73329,...,-0.088232,0.302075,-1.377155,0.736659,1.138024,1.329556,3.873212,0.070319,3.389347,3.392994
202184900076_R04C01,2.927024,-0.833248,-1.561893,-0.689921,2.775226,-2.636557,0.941312,1.624266,0.95653,3.514725,...,-0.377822,0.294278,-0.872458,0.668582,1.780835,1.256327,4.096688,-0.33414,3.277219,3.13821
202184900076_R06C01,2.334874,-0.675145,-0.769275,-2.226923,3.475557,-2.879151,1.450152,3.24755,2.881064,3.947485,...,1.961999,2.0729,-1.18314,0.832224,1.772527,0.597387,3.97231,-0.074274,3.015352,3.929728


In [9]:
# Take out test set 
x_notest, x_test, y_notest, y_test = train_test_split(x_full, y_full, stratify = df['Endometriosis_int'], test_size = 0.05)

In [10]:
x_test['Cycle_Phase_int'] = y_test

In [11]:
# Merge phenotypes and x_test 
test2 = pheno.merge(x_test, on = 'meth_id', how='right')
# Remove test from full dataset
test2.set_index('meth_id', drop=True, inplace=True) 
df = df.drop(test2.index)

In [12]:
# Split dataset into features and labels
x = df.iloc[:, 5:-4] # CPG probes
y = df['Cycle_Phase_int'] # Endo Case vs Control 

In [13]:
# Split data into train and val (80/20) stratified by cycle phase 
X_train, X_val, y_train, y_val = train_test_split(x, y, stratify=df['Endometriosis_int'], test_size=0.2)

In [14]:
# Change labels to int for linear regression 
y_train = y_train.astype('float64')

# Transpose features df for lin regress
X_train = X_train.T

In [15]:
# set up function for feature extraction using linear regression
import scipy 
from scipy.stats import linregress
def calc_slope(row):
    a = scipy.stats.linregress(y_train, row)
    #print(pd.Series(a._asdict()))
    return pd.Series(a._asdict())

In [16]:
# get train features 
cols = X_train.columns

# convert all columns to numeric
X_train = X_train[cols].apply(pd.to_numeric, errors='coerce').fillna(0)

# Perform feature extraction
feat_extract = X_train.join(X_train.apply(calc_slope,axis=1))

In [17]:
#Benjamini-Hochberg filtering p-values
feat_extract = feat_extract.sort_values(by='pvalue')
feat_extract['rank'] = feat_extract.reset_index().index + 1
feat_extract['(I/m)Q'] = (feat_extract['rank']/len(feat_extract))*0.05
feat_extract['(I/m)Q - p'] = feat_extract['(I/m)Q'] - feat_extract['pvalue']
feat_extract['nominally_sig'] = np.where(feat_extract['pvalue'] < 0.05, 1, 0)
feat_extract['BH_sig'] = np.where(feat_extract['(I/m)Q - p'] < 0, 0, 1)

In [18]:
# number of features nominally and BH significant
feat_extract['nominally_sig'].sum(), feat_extract['BH_sig'].sum()

(89852, 15522)

In [19]:
# Filter for nominally significant probes
feat_extract_NomSig = feat_extract[feat_extract['nominally_sig'] == 1]

# remove output from linear regression
feat_extract_NomSig = feat_extract_NomSig.iloc[:, :-11]

feat_extract_NomSig = feat_extract_NomSig.T

In [20]:
# merge train and phenotype df
train_df = pheno.merge(feat_extract_NomSig, left_on='meth_id', right_on=feat_extract_NomSig.index, how='right')
train_df = train_df.loc[:, ~train_df.columns.duplicated()]

# Group ESE, MSE and LSE into SE 
train_df.loc[train_df['Cycle_Phase'].isin(['ESE', 'MSE', 'LSE']), 'Cycle_Phase'] = 'SE'

train_df.set_axis(train_df['meth_id'], inplace = True)
train_df['Endometriosis_int'] = y_train

In [21]:
# Keep the same features as training set in val set
val = X_val[feat_extract_NomSig.columns]

#merge validation and pheno df
val = pheno.merge(val, left_on= 'meth_id', right_on=val.index, how='right')
val = val.loc[:, ~val.columns.duplicated()]

# Change Endo Yes/No to binary
val['Endometriosis_int'] = np.where(val['Endometriosis'] == 'Yes', 1, val['Endometriosis'])
val['Endometriosis_int'] = np.where(val['Endometriosis_int'] == 'No', 0, val['Endometriosis_int'])

# only include the columns present in train
ext_test = test2[feat_extract_NomSig.columns]

In [22]:
ext_test = pheno.merge(ext_test, on = 'meth_id', how='right')

In [23]:
# Change Endo Yes/No to binary
ext_test['Endometriosis_int'] = np.where(ext_test['Endometriosis'] == 'Yes', 1, ext_test['Endometriosis'])
ext_test['Endometriosis_int'] = np.where(ext_test['Endometriosis_int'] == 'No', 0, ext_test['Endometriosis_int'])

In [24]:
print(val.shape, ext_test.shape, train_df.shape)

(187, 89859) (50, 89859) (747, 89859)


In [25]:
# Export feature extracted train set as csv
train_df.to_csv('Endometriosis_subset_train_feature_extract_no_mamc_splitfunction_cycle_phase_stratify_by_endo5.csv')

In [26]:
# Export val df as csv
val.to_csv('Endometriosis_subset_val_feature_extract_no_mamc_splitfunction_cycle_phase_stratify_by_endo5.csv')

In [27]:
ext_test.to_csv('Endometriosis_subset_test2_feature_extract_no_mamc_splitfunction_cycle_phase_stratify_by_endo5.csv')